#### Parameter Tuning

In [1]:
#Set Tuning Parameter
k = list(range(10,50))
docConcentration = [0.05, 0.1, 0.5, 1, 5, 10]
topicConcentration = [0.05, 0.1, 0.5, 1, 5, 10]

In [ ]:
#Test Set
k = [25]
docConcentration = [[0.05]]
#docConcentration = [float(x) for x in docConcentration]
topicConcentration = [0.05]
#topicConcentration = [float(x) for x in topicConcentration]b

In [ ]:
j = 2
result_list = []
for i in k:
    for d in docConcentration:
        for t in topicConcentration:
            lda = LDA(featuresCol="features", maxIter = 100, seed=1, optimizer="online", k = i, topicConcentration = t, docConcentration = d)
            ldaModel=lda.fit(tfidf_new)
            
            #Check if it's distributed model then convert to local model
            if ldaModel.isDistributed() == True:
                ldaModel = ldaModel.toLocal()
                logperplexity = ldaModel.logPerplexity(tfidf_new)
                path = "./models"
                model_number = '{}'.format(j)
                ldaModel.save(path + 'LDAModel_'+ model_number)
                lda.save(path + 'LDA_'+ model_number)
            else:
                logperplexity = ldaModel.logPerplexity(tfidf_new)
                path = "./models"
                model_number = '{}'.format(j)
                ldaModel.save(path + 'LDAModel_'+ model_number)
                lda.save(path + 'LDA_'+ model_number)
            
            #Append results to list   
            result_list.append(tuple([j,logperplexity,i,d,t]))
            j = j+1

In [ ]:
from operator import itemgetter
result_list.sort(key=itemgetter(1))

#### Evaluate Model Performance on Brand New Data

In [ ]:
from pyspark.ml.clustering import LocalLDAModel
# from pyspark.ml.clustering import DistributedLDAModel
path = os.getcwd()
ldaModel_base = LocalLDAModel.load(path + 'LDAModel_base')
ldaModel_1 = LocalLDAModel.load(path + 'LDAModel_1')